# Sportsbook ESPN Player Handle Validation

## Step 0: Imports

In [2]:
!pip install -q dask[complete] catboost

In [3]:
import datetime
import pandas as pd
import numpy as np
import dask.dataframe as dd

from typing import Tuple, List
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import r2_score, mean_absolute_error

pd.options.mode.chained_assignment = None  # default='warn'

# notification sound
from IPython.display import Audio
notify_me = Audio(filename = 'pop_notification.wav', rate = 4410, autoplay=True)

## Step 1: Read in Data from GCS
The training data is already queried and written to GCS [here](https://console.cloud.google.com/storage/browser/non-prod-data-eng-dev-dsml/training-storage/espn-player-handle-forecast/sportsbook_retention?project=non-prod-data-eng-dev&pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))):
* Bucket: `non-prod-data-eng-dev-dsml`
* Path: `training-storage/espn-player-handle-forecast/sportsbook_retention`

In the path above, there are 2 directories, `actuals` and `features`. The `actuals` directory contains the outcome variable that we are trying to predict `y`. Specifically, this is handle and activedays for the next 7 and 14 days. The dataset is at a `user_id` x `predicted_on_date` granularity. I.e., there is 1 row per user and date.

The `features` directory contains the features (i.e., model inputs `X`) which we will use to predict each `y`. Importantly, the features are always historical (i.e., from *before* `predicted_on_date`) and the actuals are always forward looking (i.e., *on or after* the `predicted_on_date`).

### Partitioning
The dataset is stored as date-partitioned parquet files. The top-level partition is date (e.g., `/date=2024-03-01`) and within each partition, there exists individual partitions of ~100k rows of data (`train_0.parquet`, `train_1.parquet`, ..., `train_n.parquet`). Furthermore, there is an empty file in each date partition called `_SUCCESS`. This is called a success sensor, which tells us that the partition of data is complete and good to use.

You may find the partitioning to be intimidating. But no worries, below I will show you how to read in all the partitions in 1 line of code using a handy library called `dask` which we have installed above. You might ask, why do we even partition in the first place? Here are some reasons why this is ideal for a production setting:
* Since we store all the historical date partitions, on each new day, we only need to run the query for 1 day of data (as opposed to all days which is redundant)
* Since we run the query for 1 day at a time, we will avoid out-of-memory (OOM) issues on Dagster in production

Lets start by reading in the data with `dask` and converting it to a familiar `pandas` dataframe.

### Constants
In the cell below, I will set some constant values which point to the GCS location of the data. We can re-use these throughout the notebook! In python, constants are typically set in ALL CAPS by convention.

In [4]:
STORAGE_BUCKET = 'non-prod-data-eng-dev-dsml'
GCS_PATH = 'training-storage/espn-player-handle-forecast/sportsbook_retention/{query_type}'

In [5]:
# We can use python string formatting to dynamically change the value of `{query_type}` to point to either features or actuals
# For example:
GCS_PATH.format(query_type='actuals')

'training-storage/espn-player-handle-forecast/sportsbook_retention/actuals'

#### Sidebar on python strings
There are 3 types of strings in python:
* Static: `'hello world!'`
* Formattable: `'hello {thing}!'`
    * `'hello {thing}!'.format(thing='world')`
* F-strings:
    * `thing = 'world'`
    * `f'hello {thing}!'`
    
Static strings do not change. Their value is literal.
Formattable strings contain `{}` and can be changed dynamical *later*, by calling `.format()`
Finally, F-strings are formatting *immediately* using the value of other variables. 

Try some examples for yourself before proceeding to the next section!

We are now ready to read in some parquet files all at once using `dask`! There are 3 points to keep in mind when using it:
* The path should be formatted like `gs://{bucket_name}/{path_to_parquet_files}`. The `gs://` tells `dask` to look in GCS rather than local storage.
* Using `dask` (but not `pandas`) you can use the `*` as a wildcard to read in multiple parquet files in 1 line of code!
* To convert to a `pandas` df, you just need to use `dask_df.compute()` and it will be converted.

Lets try on a single date partition:

In [6]:
# First, we'll format the path to read in 1 date partition of data, 2024-03-01.
# Inside this date partition, there are 7 parquet files which we will read in all at once.

# First lets just reading the actuals and do features later.
actuals_path = GCS_PATH.format(query_type='actuals')
full_path = f'gs://{STORAGE_BUCKET}/{actuals_path}/date=2024-03-01/train_*.parquet'
print(full_path)

gs://non-prod-data-eng-dev-dsml/training-storage/espn-player-handle-forecast/sportsbook_retention/actuals/date=2024-03-01/train_*.parquet


Notice the `*` in `train_*.parquet`. This tells `dask` that any file matching the pattern `train_{ANY_VALUE}.parquet` should be read in. Let's try it!

In [7]:
dask_df = dd.read_parquet(full_path)
dask_df

,user_id,predicted_on_date,activedays_7_days,activedays_14_days,handle_7_days,handle_14_days,date
npartitions=7,,,,,,,
,string,date32[day][pyarrow],int64,int64,float64,float64,category[known]
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


Looks good, we have read in 7 partitions of data in 1 line of code! Now finally, lets convert to a pandas dataframe.

In [8]:
df_actuals = dask_df.compute()
df_actuals

,user_id,predicted_on_date,activedays_7_days,activedays_14_days,handle_7_days,handle_14_days,date
0,8833bc88-299b-4e93-9c27-88db6c3f48da,2024-03-01,0,0,0.0,0.00,2024-03-01
1,ba008e31-ea16-4027-9673-02a9365b73c2,2024-03-01,1,2,0.0,76.87,2024-03-01
2,f5f0b708-902f-4120-b038-f7b4e6e6eab8,2024-03-01,0,0,0.0,0.00,2024-03-01
3,6312af61-9459-464f-820b-6ed8ecd3e310,2024-03-01,1,4,37.0,139.00,2024-03-01
4,6d7cbd14-e2d6-40fc-b6bf-2157dfb89d98,2024-03-01,5,11,55.0,94.00,2024-03-01
...,...,...,...,...,...,...,...
82401,505cc65f-141d-4f45-b0f1-be04af192a19,2024-03-01,0,0,0.0,0.00,2024-03-01
82402,828e8f76-5992-4e84-8c3e-23bdff7d669c,2024-03-01,0,0,0.0,0.00,2024-03-01
82403,e5f07afa-7842-4ce3-bd5a-8fbe3b2d5809,2024-03-01,0,0,0.0,0.00,2024-03-01
82404,59568692-6517-4e35-9f64-b4f869868655,2024-03-01,0,0,0.0,0.00,2024-03-01


### Exercise 1: Read in all parquet files for actuals and features. Join them using pandas `df.merge()` to one big dataframe.

In your first exercise, you will complete the code in the function below. Any elipses you see `...` should be replaced with your own code.

Above, I have shown you how to read in all parquet files in 1 date partition. Below, you will read in all parquet files across **all** date partitions.
* **Hint**: You can use more than 1 wildcard `*` in a `dask` path :)

Once you've done this with actuals, do the same with features. Finally, you will join actuals with features using pandas `.merge()`. See the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).
* **Hint**: Remember, the dataset is 1 row per *user and predicted_on_date* so think about what join key(s) are appropriate to use.

In [9]:
def load_and_join_actuals_with_features() -> pd.DataFrame:
    """
    Loads all actuals and features partitions in GCS using dask and
    converts to pandas dataframe. Joins actuals with features and returns
    the joined pandas dataframe.
    
    Returns
    -------
    pd.DataFrame
        Dataframe of grain 1 row per user and predicted_on_date, with
        all actuals (y) and features (X)
    """
    # Read in actuals df using dask and convert to pandas df
    actuals_path = GCS_PATH.format(query_type='actuals')
    full_path = f'gs://{STORAGE_BUCKET}/{actuals_path}/date=*/train_*.parquet'
    df_actuals = dd.read_parquet(full_path).compute()
    
    # Read in features df using dask and convert to pandas df
    features_path = GCS_PATH.format(query_type='features')
    full_path = f'gs://{STORAGE_BUCKET}/{features_path}/date=*/train_*.parquet'
    df_features = dd.read_parquet(full_path).compute()
    
    # Join actuals with features and return
    merged_df = df_features.merge(df_actuals, how = 'inner', on = ['user_id', 'predicted_on_date'])
    
    return merged_df

Once you've completed the function above, lets try it out!
If your df contains ~58M rows, you are looking good. You can check all the columns and datatypes using `df.dtypes`.

In [9]:
df = load_and_join_actuals_with_features()
df.sample(3)

,user_id,predicted_on_date,days_since_first_bet,activedays_prev_1_day,activedays_prev_3_to_2_days,activedays_prev_7_to_4_days,activedays_prev_15_to_8_days,activedays_prev_31_to_16_days,activedays_prev_63_to_32_days,handle_prev_1_day,...,n_ncaaf_legs,crs_pregame,crs_live,eod_cash_balance,date_x,activedays_7_days,activedays_14_days,handle_7_days,handle_14_days,date_y
17396924,aed96d96-e87d-4da2-b9fa-b4221564c803,2024-03-28,253,0,0,0,0,0,4,0.0,...,1,1.0,1.0,0.00,2024-03-28,1,1,0.0,0.0,2024-03-28
50342628,e4befc9e-d07d-4929-b4f8-4e9d36d5002d,2024-05-26,191,0,0,0,0,1,1,0.0,...,0,1.0,1.0,0.00,2024-05-26,0,0,0.0,0.0,2024-05-26
20076898,8877493b-c3db-46fd-9318-d5032f431167,2024-04-01,949,0,0,0,2,0,2,0.0,...,0,1.0,1.0,1425.69,2024-04-01,2,3,110.0,170.0,2024-04-01


In [10]:
df.dtypes

user_id                               string[pyarrow]
predicted_on_date                date32[day][pyarrow]
days_since_first_bet                            int64
activedays_prev_1_day                           int64
activedays_prev_3_to_2_days                     int64
activedays_prev_7_to_4_days                     int64
activedays_prev_15_to_8_days                    int64
activedays_prev_31_to_16_days                   int64
activedays_prev_63_to_32_days                   int64
handle_prev_1_day                             float64
handle_prev_3_to_2_days                       float64
handle_prev_7_to_4_days                       float64
handle_prev_15_to_8_days                      float64
handle_prev_31_to_16_days                     float64
handle_prev_63_to_32_days                     float64
n_straights                                     int64
pct_straights                                 float64
n_parlays                                       int64
pct_parlays                 

In [11]:
df.shape #58,751,150

(61456405, 39)

In [12]:
notify_me

## Step 2: Train-Test Split

Now that you've read in and merged the actuals and features, next we will setup our train-test split. As you may know, in machine learning, we typically train on 1 subset of the data, and test on a hold-out sample of data to assess accuracy. The model is not allowed to see the data in the test set during training, which gives us an sense of how the model will generalize its learning on never-before-seen data.

In many ML problems, we would use `sklearn`'s `train_test_split`. However, it is not appropriate for the current project. This is because our project has a time factor and our dataset is 1 row per user **and predicted_on_date**. If we simply use `sklearn`'s `train_test_split`, we might train on the same user's data in the future, and predict on that same user in the past. This is not an accurate representation of the model's accuracy in production.

Not to worry though, we will write our own train-test split function, which is time-aware.

Specifically, given a single test date (for this exercise we will use `test_date` = `2024-06-01`):
* All rows of data with a `predicted_on_date` < `test_date` will be used for training.
* Data exactly on `test_date` will be used for testing / validation.

In future exercises (outside of this notebook), we will scale this up to multiple test dates, but for now, lets just focus on the 1.

In [13]:
test_date = pd.Timestamp('2024-06-01').date()
print(f'Test date: {test_date}')

Test date: 2024-06-01


### Exercise 2: Custom time-aware train-test split

Complete the function below for a custom time-aware train test split. First you will filter the df for all data prior to `test_date` and store this in `df_train`.
Next you will filter for data exactly on `test_date` and this will be stored in `df_test`. You will return both from the function.

**Note**: When you return more than 1 variable from a python function, it is returned in a `tuple`. This is basically a list (but immutable). Read more about it [here](https://www.w3schools.com/python/python_tuples.asp)

If you are unfamiliar with how to filter a pandas df, think about it like a `WHERE` clause in `SQL`.
* **Hint**: You will need to use the `df.loc[]` operator, read more about it [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) and also [here](https://towardsdatascience.com/effective-data-filtering-in-pandas-using-loc-40eb815455b6)

In [14]:
def time_aware_train_test_split(df: pd.DataFrame,
                                test_date: datetime.date) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Given an input df containing all data and a column `predicted_on_date`, perform a
    time-aware train test split using the input `test_date`. All data prior to `test_date`
    is used for training. All data exactly on the `test_date` is used for testing.
    
    Params
    ------
    df : pd.DataFrame
        df of all data, features and actuals across all dates.
    test_date: datetime.date
        The test date used for splitting
        
    Returns
    -------
    Tuple[pd.DataFrame, pd.Dataframe]
        The training set and the test set, respectively
    """
    # All data prior to `test_date` are used for training
    df_train = df.loc[df["predicted_on_date"] < test_date]
    
    # All data exactly on `test_date` is used for testing
    df_test = df.loc[df["predicted_on_date"] == test_date]
    
    return df_train, df_test

Lets try it out! Your training dataset should have a max `predicted_on_date` of `2024-05-31` and your test dataset should only have `predicted_on_date` of `2024-06-01`.

Please verify this for yourself and if that is true, we are good to move on!

In [15]:
df_train, df_test = time_aware_train_test_split(df, test_date)

In [16]:
df_train.sample(3)

,user_id,predicted_on_date,days_since_first_bet,activedays_prev_1_day,activedays_prev_3_to_2_days,activedays_prev_7_to_4_days,activedays_prev_15_to_8_days,activedays_prev_31_to_16_days,activedays_prev_63_to_32_days,handle_prev_1_day,...,n_ncaaf_legs,crs_pregame,crs_live,eod_cash_balance,date_x,activedays_7_days,activedays_14_days,handle_7_days,handle_14_days,date_y
30559272,43503f7c-5005-4470-9d47-f916e28ad529,2024-04-17,869,0,0,0,1,9,11,0.0,...,3,1.0,1.0,0.00,2024-04-17,0,0,0.0,0.0,2024-04-17
2717485,57f934f4-122a-45d3-8235-34fcb43d72c1,2024-03-05,100,0,0,0,0,1,1,0.0,...,3,1.0,1.0,0.70,2024-03-05,0,0,0.0,0.0,2024-03-05
8735391,d8e6a8d9-d47c-46c4-9f0f-f99d95b4316b,2024-03-14,1100,1,2,4,2,3,20,3908.0,...,101,6.0,6.0,0.41,2024-03-14,7,8,11551.0,11551.0,2024-03-14


In [17]:
df_test.sample(3)

,user_id,predicted_on_date,days_since_first_bet,activedays_prev_1_day,activedays_prev_3_to_2_days,activedays_prev_7_to_4_days,activedays_prev_15_to_8_days,activedays_prev_31_to_16_days,activedays_prev_63_to_32_days,handle_prev_1_day,...,n_ncaaf_legs,crs_pregame,crs_live,eod_cash_balance,date_x,activedays_7_days,activedays_14_days,handle_7_days,handle_14_days,date_y
52756178,01c80061-ae32-491f-bfdd-d40c25cb0643,2024-06-01,449,0,2,4,2,12,14,0.0,...,8,1.0,1.0,339.76,2024-06-01,3,7,170.0,410.00,2024-06-01
52647452,f84c765b-8a92-4c3a-b6b6-b575ba5a915a,2024-06-01,98,0,0,0,2,4,15,0.0,...,0,1.0,1.0,430.00,2024-06-01,1,5,30.0,91.32,2024-06-01
52532752,b3ffe8b7-139d-4a37-9735-ab25d2c0bca5,2024-06-01,344,0,0,0,1,0,2,0.0,...,0,1.0,1.0,0.00,2024-06-01,0,0,0.0,0.00,2024-06-01


In [18]:
notify_me

## Step 3: Train Baseline Model

Now that we have our data split into train and test, we can train our first model! For these types of problems (supervised regression: [see this](https://cloud.google.com/discover/supervised-vs-unsupervised-learning)), it can be beneficial to start simple and scale up. The simplest model we can train is called a "baseline model". Often, this is a model with just 1 or 2 features, which allows us to establish a baseline that we can improve on!

The reason it is beneficial to start simple: Lets say we begin with the opposite, we throw all the features into our first model and it performs very badly (negative R^2). It would be harder for us to asses what went wrong. With a baseline model, we can slowly add 1 feature and make one change at a time.

For our first model, we will try to predict `y = 'handle_7_days'` using 2 features: `X = ['handle_prev_1_day', 'handle_prev_3_to_2_days']`

### Exercise 3: Baseline Model

Complete the function below `train_catboost_model`, as well as the cell below that which calls the function. 

To complete the function `train_catboost_model`, you'll need to know how to extract columns from a pandas dataframe. Read more about it [here](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html)!

You should call the function with the appropriate values for the 3 input params. 
* **Hint**: Remember, we are trying to predict `y = 'handle_7_days'` using 2 features: `X = ['handle_prev_1_day', 'handle_prev_3_to_2_days']`

In [19]:
def train_catboost_model(df_train: pd.DataFrame, y_name: str, X_names: List[str]) -> CatBoostRegressor:
    """
    Train a catboost model to predict df[y_name] using df[X_Names]
    
    Params
    ------
    df_train : pd.DataFrame
        Training dataset of features and actuals
    y_name : str
        Name of column containing the outcome variable
    X_names : List[str]
        List of name(s) of column(s) containing the features
        
    Returns
    -------
    CatboostRegressor
        Trained catboost regressor model
    """
    # Extract outcome variable and features from df_train
    y_train = df_train[y_name]
    X_train = df_train[X_names]
    
    # Instantiate model and pool dataset
    model = CatBoostRegressor(eval_metric="RMSE", verbose=300)
    train_pool = Pool(X_train, y_train)
    
    # Train model
    model.fit(train_pool)
    
    return model

Lets try it out! (FYI training should take ~18 minutes). If the model ends training with a learn loss of RMSE = XXXX, you should be good!

In [20]:
model = train_catboost_model(df_train = df_train, y_name = 'handle_7_days', X_names = ['handle_prev_1_day', 'handle_prev_3_to_2_days'])

Learning rate set to 0.227986
0:	learn: 5448.5964944	total: 993ms	remaining: 16m 32s
300:	learn: 5212.6475449	total: 4m 2s	remaining: 9m 22s
600:	learn: 5212.2697758	total: 8m 55s	remaining: 5m 55s
900:	learn: 5212.0926899	total: 12m 58s	remaining: 1m 25s
999:	learn: 5212.0486629	total: 14m 17s	remaining: 0us


In [21]:
notify_me

## Step 4: Evaluate

In the final step of this notebook, we will evaluate the model's accuracy. Evaluation involves 2 steps:
1. Generate predictions on the test set
2. Score accuracy by comparing predictions to test set actuals

For scoring, we will use 2 metrics, which you have seen before: R^2 and mean absolute error (MAE).
* [R^2](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html): Ranges from -Inf to 1, where 1 is perfect prediction. 
* [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html): This is in raw units. So MAE = 100 means on average, your prediction is 100$ off from the actual.

For both metrics, we will use the implementation from `sklearn`, please read the linked documentation above on how to use `r2_score` and `mean_absolute_error`!

### Exercise 4: Evaluate

Complete the function below. To generate predictions, check out the CatBoost documentation [here](https://catboost.ai/en/docs/concepts/python-usages-examples) and [here](https://catboost.ai/en/docs/concepts/python-reference_catboostregressor_predict)

Next, compute the 2 accuracy metrics and call the function using the appropriate input params!

In [22]:
def evaluate_accuracy(model: CatBoostRegressor,
                      df_test: pd.DataFrame,
                      y_name: str,
                      X_names: List[str]) -> Tuple[float, float, np.array]:
    """
    Given a trained model and a test set, evaluate model accuracy.
    First we generate predictions using df_test[X_names]. Then we compare
    the predictions to df_test[y_name] to calculate R^2 and MAE.
    
    Params
    ------
    model : CatBoostRegressor
        Trained Catboost model
    df_test : pd.DataFrame
        Test dataset containing features and actuals
    y_name : str
        Name of column containing the outcome variable
    X_names : List[str]
        List of name(s) of column(s) containing the features
        
    Returns
    -------
    Tuple[float, float]
        The R^2 and MAE metrics, respectively
    np.array
        The raw predictions
    """
    # Generate predictions
    predictions = model.predict(df_test[X_names])
    
    # Compute metrics
    r2 = r2_score(df_test[y_name], predictions)
    mae = mean_absolute_error(df_test[y_name], predictions)
    
    return r2, mae, predictions

In [23]:
r2, mae, predictions = evaluate_accuracy(model = model, df_test = df_test, y_name = ['handle_7_days'], X_names = ['handle_prev_1_day', 'handle_prev_3_to_2_days'])

Lets take a look at our results! If your R^2 is ~0.20 and your MAE is ~160, then you are looking good!

In [24]:
print(f'R^2: {r2}, MAE: {mae}')

R^2: 0.2125960373500544, MAE: 162.70269472088384


Another good sanity check to perform is to sample raw users and see if the prediction is actually close to actuals.

Lets tack the predictions on to `df_test` and take a look. You can run the cell below many times to sample different users:

In [25]:
df_test['predictions'] = predictions
df_test[['user_id', 'handle_7_days', 'predictions']].sample(10)

,user_id,handle_7_days,predictions
52673401,a7abade4-12ba-452a-8e9d-5054ca7b62c2,56.78,110.654752
52728200,9436be43-0b33-4f87-9a50-1269555a6642,0.00,34.450741
52748576,70403e36-5708-4e6b-a562-679cbd4be189,31.34,282.467132
52707235,a2b5d8d7-41d5-4658-9d8f-549df87290d6,0.00,34.450741
52580231,47a6354d-ca0a-4e2a-9950-01a6498bb478,9.00,34.450741
52548189,21f62d15-94fa-4880-b798-03b7b57aa95d,0.00,34.450741
52706496,cea63274-164b-4396-969b-a358d69906c1,145.00,135.020582
52535182,eb1c9948-1b31-4438-8b6b-657c5ab759d9,0.00,34.450741
52497230,988d4d01-b5f6-41cc-b7d7-15dea736abee,0.00,34.450741
52637024,4161e3ac-d51a-4f86-ba4e-24a20442f2e4,194.42,298.967959


In [26]:
df_test[['user_id', 'handle_7_days', 'predictions', 'handle_prev_1_day', 'handle_prev_3_to_2_days']].describe()

,handle_7_days,predictions,handle_prev_1_day,handle_prev_3_to_2_days
count,3.804210e+05,380421.000000,380421.000000,380421.000000
mean,1.592961e+02,157.495436,23.562461,44.221960
std,3.786122e+03,1997.194633,626.373296,1374.261252
min,0.000000e+00,8.682412,0.000000,0.000000
25%,0.000000e+00,34.450741,0.000000,0.000000
50%,0.000000e+00,34.450741,0.000000,0.000000
75%,2.000000e+01,34.450741,0.000000,0.000000
max,1.389392e+06,136275.450175,210561.140000,514127.000000


In [27]:
notify_me

Results look okay, with lots of room to improve. We are definitely doing better than chance but looks like the model is showing some biases that are blatently incorrect... can you identify what this pattern is?

## Conclusion

In this notebook, we trained a baseline model with 2 features on a single test date. Results are not bad, but a lot to improve on! That's okay, its just a baseline ;).

In the next session / notebook, we will cover the following:
* Interpret feature importance
* Add more features to the model
* Use a for-loop to evaluate multiple test dates!
* Plotting your results